In [57]:
import os
import pandas as pd
import csv
import re

In [62]:
def match_label(to_match, to_not_match="", labels=None):
    match_regex_str = f"^(?=.*({to_match}))"
    if to_not_match != "":
        match_regex_str = f"{match_regex_str}(?!.*({to_not_match}))"
    match_regex = re.compile(
        f"{match_regex_str}(?!.*(interest|desire|prefer|open)).*",
        flags=re.IGNORECASE
    )
    desired_regex = re.compile(
        f"{match_regex_str}.*", 
        flags=re.IGNORECASE
    )
    # matched_label = None
    desired_label = None
    for label in labels:
        if match_regex.match(label):
            return label
        if desired_regex.match(label):
            desired_label = label
    return desired_label

In [63]:
def detect_interested_label(labels: list[str]):
    """
    Detect labels of interested, i.e ["name or email (as ID)", "title/role", "department/area/function", "location"]
    """
    title_label = match_label(to_match="title|role|position|domain", labels=labels)
    function_label = match_label(to_match="department|area|function|team|discipline", labels=labels)
    city_label = match_label(to_match="city", labels=labels)
    country_label = match_label(to_match="country", labels=labels)
    location_label = match_label(to_match="location", labels=labels)
    name_label = match_label(to_match="name", labels=labels)
    location_label = city_label or country_label or location_label
    label_list = [name_label, title_label, function_label, location_label]
    return label_list
    # label_map = {
    #     "name": name_label,
    #     "title": title_label,
    #     "function": function_label,
    #     "location": location_label
    # }
    # return label_map

In [64]:
# Create an empty list to store the first row of each CSV file
first_rows = []

# if the first column more than 50 character it is not likely to be column
len_character_threshold = 50

# Define a regular expression pattern to match HTML tags
html_tag_pattern = re.compile(r'<\s*(html|head|body|table)\b[^>]*>', re.IGNORECASE)

invalid_first_row_count = 0
invalid_file_count = 0
employee_count = 0

def clean_csv(input_file, output_dir):
    rows = []
    labels = []
    # print(f"Opening file: {input_file}")
    # Open the CSV file
    with open(input_file, 'r', encoding='utf-8') as csvfile:
        # Read the entire contents of the CSV file as a string
        file_content = csvfile.read()
        
        # Check if the file contains any HTML tags
        if re.search(html_tag_pattern, file_content):
            # Skip to the next file if the file contains HTML content
            print(f"skipping: HTML element found in file {input_file}")
            return None
        
        csvfile.seek(0)  # move file pointer to start of file
        # Create a CSV reader object
        reader = csv.reader(csvfile, delimiter=',')
        try:
            # Get the first row of the CSV file
            while len(labels) == 0:
                current_row = next(reader)
                if (len(current_row[0]) >= len_character_threshold) \
                        or (len(current_row) > 1 and len(current_row[1]) >= len_character_threshold):
                    # print("skipping: too many characters in first row cell")
                    # print(current_row[0])
                    continue

                row_text = "|".join(current_row).lower()
                if row_text.find("name") != -1 or \
                    row_text.find("nome") != -1 or \
                    row_text.find("nombre") != -1:
                    labels = current_row

            print(f"label row found: {labels}")
            interested_labels = ["||".join(labels)]
            interested_labels.extend(detect_interested_label(labels))
            return interested_labels
        
            while False:
                row = next(reader)
                if len(row) <= len(labels):
                    if "".join(row).strip() == "":
                        print("skipping empty row")
                        continue
                    rows.append(row)
                else:
                    rows.append(row[:len(labels)])
        except StopIteration as e:
            print(f"Reached end of file: {input_file}")
            # Skip this file if it has less than one row
            if len(labels) > 0 and len(rows) > 0:
                df = pd.DataFrame(rows, columns=labels)
                output_file = input_file.split('/')[-1].split('.')[0]
                output_path = f"{output_dir}/{output_file}-cleaned.csv"
                print(f"writing output file to {output_path}")
                df.to_csv(output_path)
                return output_path
            return None
        except Exception as e:
            print(f"Exception error file not valid: {e} in {input_file}")
            return None

In [65]:
folder_path = "../../employee_csv_20230323/"
label_data = []
invalid_files = []
for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        interested_labels = clean_csv(input_file=f"{folder_path}/{filename}", output_dir="output")
        d = [filename]
        if interested_labels is not None:
            d.extend(interested_labels)
            label_data.append(d)
        else:
            invalid_files.append(filename)
        #     d.extend(["", ""])
        #     label_data.append()
label_df = pd.DataFrame(data=label_data, columns=["file_name", "labels", "name_label", "title_label", "function_label", "location_label"])
print(label_df.shape)
print(label_df[label_df["function_label"].isna()].shape)

label row found: ['First & Last Name', 'Personal Email Address', 'LinkedIn Profile Link', 'Role at DoorDash', 'Expertise / Roles Interested In', 'Preferred Location', 'Function', '', '', '', '', '']
label row found: ['Name', 'Role', 'LinkedIn', 'Location', 'Country', 'Visa Support (Y/N)', 'Years of Experience', 'Expertise/Roles Interested', 'DeleteRequestRecieved', 'US/Non US', 'Visa_Flag', 'f1', 'f2', 'US/Non US Calc']
label row found: ['Name', 'Previous Title', 'Current Location', 'Preferred Work Mode', 'LinkedIn Page', 'Last updated: Wednesday, 12/7/22 @ 2:12pm ET']
label row found: ['', 'Complete Name', 'Job title at SSENSE', 'General Discipline', 'Current Location', 'Work Mode Preference', 'LinkedIn Link']
Reached end of file: ../../employee_csv_20230323//Meta #3 laid-off employees_1.csv
Reached end of file: ../../employee_csv_20230323//Nomad Health laid-off employees_0.csv
label row found: ['Date', 'Name', 'Function', 'Job title / role', 'Current location', 'Professional profile 

In [26]:
def extract_csv(input_file):
    df = pd.read_csv(input_file)
    return df
    found = [i for i in range(0, len(df.columns)) \
        if df.columns[i]=='title' or df.columns[i]=='location' or df.columns[i]=='function']
    if len(found) == 3:
        extracted_df = df[['title','location','function']]
        return extracted_df
    return None

In [39]:
def combine_csv(inputs, output_dir):
    combined_df = pd.DataFrame(columns=['name','title','function','location'])
    combined_file_path = f"{output_dir}/employee_merged_csv.csv"
    for f in inputs:
        csv_df = extract_csv(f)
        combined_df = pd.concat([combined_df, csv_df], ignore_index=True)
    combined_df.to_csv(combined_file_path)
    return combined_file_path

In [42]:
folder_path = "../../employee_cleaned_csv_20230323/"
inputs = list([f"{folder_path}{f}" for f in os.listdir(folder_path)])
combined_df = combine_csv(
    inputs=inputs[:10],
    output_dir="../../out"
)

FileNotFoundError: [Errno 2] No such file or directory: '../../employee_cleaned_csv_20230323/'